## Dataset [Shoulder Implant X-Ray Manufacturer Classification Data Set](https://archive.ics.uci.edu/ml/datasets/Shoulder+Implant+X-Ray+Manufacturer+Classification)

##Data preprocessing

In [7]:
import os
from sklearn import preprocessing
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from keras.applications.resnet_v2 import ResNet50V2
from keras.layers import Dropout, Dense, Input
from keras.models import Model, load_model
from sklearn.metrics import classification_report, confusion_matrix
from pprint import pprint 
from sklearn.metrics import confusion_matrix, classification_report
import re


dir = 'Classified_Dataset/_Data'
                                                                  
                                                                                    
Relaxation_filepaths = [
    os.path.join(dir, file_name)
    for file_name in os.listdir(os.path.join(dir))
    if 'Relaxation' in file_name
]                                                                                  


Game_filepaths = [
    os.path.join(dir, file_name)
    for file_name in os.listdir(os.path.join(dir))
    if 'Game' in file_name
] 


pprint(Relaxation_filepaths, width=1)
pprint(Game_filepaths, width=1)



Relaxation_filepaths_train = Relaxation_filepaths[:int(len(Relaxation_filepaths)*0.8)]
Relaxation_filepaths_test = Relaxation_filepaths[int(len(Relaxation_filepaths)*0.8):]


Game_filepaths_train = Game_filepaths[:int(len(Game_filepaths)*0.8)]
Game_filepaths_test = Game_filepaths[int(len(Game_filepaths)*0.8):]


Train_data_filepaths = Relaxation_filepaths_train + Game_filepaths_train
Test_data_filepaths = Relaxation_filepaths_test + Game_filepaths_test
print(len(Train_data_filepaths))
print(len(Test_data_filepaths))

['Classified_Dataset/_Data/2024_06_28_Ahmad-Tarmo_Relaxation_2_5.png',
 'Classified_Dataset/_Data/2024_06_28_Markus-Tarmo_Relaxation_1_14.png',
 'Classified_Dataset/_Data/2024_06_28_Markus-Tarmo_Relaxation_4_13.png',
 'Classified_Dataset/_Data/2024_07_03_Matin-Ahmad_Relaxation_3_11.png',
 'Classified_Dataset/_Data/2024_06_28_Markus-Tarmo_Relaxation_1_1.png',
 'Classified_Dataset/_Data/2024_08_26_JariT-Tarmo_Relaxation_2_21.png',
 'Classified_Dataset/_Data/2024_07_03_Matin-Ahmad_Relaxation_4_9.png',
 'Classified_Dataset/_Data/2024_08_26_JariT-Tarmo_Relaxation_3_5.png',
 'Classified_Dataset/_Data/2024_07_03_Matin-Ahmad_Relaxation_3_19.png',
 'Classified_Dataset/_Data/2024_06_28_Markus-Tarmo_Relaxation_2_2.png',
 'Classified_Dataset/_Data/2024_07_03_Matin-Tarmo_Relaxation_1_9.png',
 'Classified_Dataset/_Data/2024_06_28_Ahmad-Tarmo_Relaxation_1_16.png',
 'Classified_Dataset/_Data/2024_06_28_Ahmad-Tarmo_Relaxation_1_25.png',
 'Classified_Dataset/_Data/2024_06_28_Markus-Tarmo_Relaxation_1_6.

In [8]:
Train_data_label = []
for filepath in Train_data_filepaths:
    t = filepath.split('.')[0].rsplit('/')[2]
    label = t.split('_')[4]
    Train_data_label.append(label)
    
    
Test_data_label = []
for filepath in Test_data_filepaths:
    t = filepath.split('.')[0].rsplit('/')[2]
    label = t.split('_')[4]
    Test_data_label.append(label)
    
    
Train_df = pd.DataFrame({'Train_data_filepaths':Train_data_filepaths,'Train_data_label':Train_data_label})
Test_df = pd.DataFrame({'Test_data_filepaths':Test_data_filepaths,'Test_data_label':Test_data_label})

Train_df

,Train_data_filepaths,Train_data_label
0,Classified_Dataset/_Data/2024_06_28_Ahmad-Tarm...,Relaxation
1,Classified_Dataset/_Data/2024_06_28_Markus-Tar...,Relaxation
2,Classified_Dataset/_Data/2024_06_28_Markus-Tar...,Relaxation
3,Classified_Dataset/_Data/2024_07_03_Matin-Ahma...,Relaxation
4,Classified_Dataset/_Data/2024_06_28_Markus-Tar...,Relaxation
...,...,...
2048,Classified_Dataset/_Data/2024_07_03_Matin-Ahma...,Game
2049,Classified_Dataset/_Data/2024_06_28_Ahmad-Tarm...,Game
2050,Classified_Dataset/_Data/2024_07_03_Matin-Ahma...,Game
2051,Classified_Dataset/_Data/2024_06_28_Markus-Tar...,Game


In [9]:
PIL.Image.open(str(Train_data_filepaths[0]))

In [10]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.).flow_from_dataframe(
    dataframe=Train_df,
    x_col='Train_data_filepaths', # Relative file paths
    y_col='Train_data_label',     # Labels
    target_size=(10, 91),         # Resize images to 10x91
    color_mode='grayscale',       # Grayscale images
    class_mode='categorical',     # Categorical labels
    batch_size=32,                # Batch size
    shuffle=True,                 # Shuffle data
    seed=124                      # Reproducibility
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255.).flow_from_dataframe(
    dataframe=Test_df,
    x_col='Test_data_filepaths',
    y_col='Test_data_label',
    target_size=(10, 91),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=124)

Found 2053 validated image filenames belonging to 2 classes.
Found 514 validated image filenames belonging to 2 classes.


In [11]:
print(Train_df.head())


                                Train_data_filepaths Train_data_label
0  Classified_Dataset/_Data/2024_06_28_Ahmad-Tarm...       Relaxation
1  Classified_Dataset/_Data/2024_06_28_Markus-Tar...       Relaxation
2  Classified_Dataset/_Data/2024_06_28_Markus-Tar...       Relaxation
3  Classified_Dataset/_Data/2024_07_03_Matin-Ahma...       Relaxation
4  Classified_Dataset/_Data/2024_06_28_Markus-Tar...       Relaxation


In [12]:
plt.figure(figsize=(20, 15))
images, encoded_labels = next(train_generator)
labels = np.argmax(encoded_labels, axis=1)

# ✅ Define key_list before using it
# key_list = {v: k for k, v in train_generator.class_indices.items()}
key_list = {0: "Relaxation", 1: "Game"}

key_list



{0: 'Relaxation', 1: 'Game'}

<Figure size 2000x1500 with 0 Axes>

##Xray Manufacturer Classification using Convolutional Neural Networks(CNN)

In [13]:
for image_batch, labels_batch in train_generator:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

# ✅ Get the number of classes correctly
num_classes = len(train_generator.class_indices)  

model = Sequential([
  layers.Rescaling(1./255, input_shape=(10, 91, 1)),  # ✅ Update for grayscale images
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)  # ✅ Use corrected num_classes
])





model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

(32, 10, 91, 1)
(32, 2)


/home/ahmad/anaconda3/lib/python3.11/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 10, 91, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 10, 91, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 5, 45, 16)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 5, 45, 32)      │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 2, 22, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 2, 22, 64)      │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 1, 11, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 704)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        90,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 113,794 (444.51 KB)

 Trainable params: 113,794 (444.51 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
epochs=120
history = model.fit(train_generator,epochs=epochs)

Epoch 1/120


/home/ahmad/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2025-02-01 09:56:12.405560: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 23068800 exceeds 10% of free system memory.
2025-02-01 09:56:12.405715: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 23068800 exceeds 10% of free system memory.


 4/65 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - accuracy: 0.6732 - loss: 0.6861

2025-02-01 09:56:12.460845: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 23068800 exceeds 10% of free system memory.
2025-02-01 09:56:12.461122: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 23068800 exceeds 10% of free system memory.
2025-02-01 09:56:12.510522: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 23068800 exceeds 10% of free system memory.


65/65 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - accuracy: 0.8308 - loss: 0.5308
Epoch 2/120
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.8362 - loss: 0.4495
Epoch 3/120
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.8358 - loss: 0.4695
Epoch 4/120
65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.8524 - loss: 0.4252
Epoch 5/120
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.8474 - loss: 0.4313
Epoch 6/120
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.8497 - loss: 0.4270
Epoch 7/120
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.8540 - loss: 0.4225
Epoch 8/120
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.8450 - loss: 0.4383
Epoch 9/120
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.8597 - loss: 0.4067
Epoch 10/120
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.8439 - loss: 0.4326
Epoch 11/120
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8344 - loss: 0.4531
Epoch 12/120
65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy:

65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9401 - loss: 0.1511
Epoch 69/120
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9439 - loss: 0.1452
Epoch 70/120
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.9482 - loss: 0.1429
Epoch 71/120
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9483 - loss: 0.1348
Epoch 72/120
65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.9417 - loss: 0.1485
Epoch 73/120
65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.9529 - loss: 0.1248
Epoch 74/120
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.9537 - loss: 0.1152
Epoch 75/120
65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.9573 - loss: 0.1194
Epoch 76/120
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9427 - loss: 0.1395
Epoch 77/120
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9524 - loss: 0.1192
Epoch 78/120
65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.9504 - loss: 0.1243
Epoch 79/120
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - a

In [15]:
y_pred_probs = model.predict(test_generator)  # Get predicted probabilities
y_pred = np.argmax(y_pred_probs, axis=1)  # Convert to class labels

# ✅ Get True Labels
y_true = test_generator.classes  # Actual labels from generator
class_labels = list(test_generator.class_indices.keys())  # Class names

# ✅ Generate Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", cm)

# ✅ Extract TP, TN, FP, FN (For Binary Classification)
if cm.shape == (2,2):  # Ensure binary classification
    TP = cm[0, 0]
    FN = cm[0, 1]
    FP = cm[1, 0]
    TN = cm[1, 1]

    # Compute Metrics
    sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
    accuracy = (TP + TN) / (TP + TN + FP + FN)

    print(f"✅ Sensitivity (Recall): {sensitivity:.4f}")
    print(f"✅ Specificity: {specificity:.4f}")
    print(f"✅ Accuracy: {accuracy:.4f}")


17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step
Confusion Matrix:
 [[382  51]
 [ 71  10]]
✅ Sensitivity (Recall): 0.8822
✅ Specificity: 0.1235
✅ Accuracy: 0.7626
